Session就是用户使用Tensorflow时的交互式接口，它被实现为Session类。Session类提供了run()方法来执行计算图。用户给run（）函数传入需要计算的节点，同时提供输入的数据，Tensorflow就会自动寻找所有需要计算的节点并按依赖顺序执行它们。

在大多数情况下，我们只会创建一次计算图，然后反复执行整个计算图或是其中的一部分子图（Subgraph）

使用会话的方式有两种，第一种（标准方式）：明确调用会话生成函数和会话关闭函数——通过Session类的Session()构造函数创建会话类实例，最后通过Session类的close()函数关闭会话释放资源

In [1]:
import tensorflow as tf

E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dty

In [2]:
a = tf.constant([1.1,1.2],name = 'a')
b = tf.constant([1.9,1.8],name = 'b')
result = a+b

sess = tf.Session() #用Session类的构造函数创建一个会话
sess.run(result)


array([3., 3.], dtype=float32)

In [3]:
sess.close()

这就是标准方式，这种方式运行会话时，必须明确调用Session.close()函数来关闭会话，同时释放资源。资源泄露通常是一件比较麻烦的事。

第二种：使用with/as环境上下文管理器。with/as语句的设计是为了和支持环境管理器协议的环境管理器对象一起工作。

Session的第二种使用方式就是运用了python的环境上下文管理器机制，只要将所有的计算放在with的内部即可。当上下文管理器退出时，就会自动释放所有的资源。这样不仅可以解决异常退出时资源释放的问题，也可以解决忘记调用Session.close()函数而产生的资源泄露的问题。

In [2]:
a = tf.placeholder(tf.float32,shape=(2),name='input')
b = tf.placeholder(tf.float32,shape=(2),name='input')
result = a + b

with tf.Session().as_default() as sess:
    sess.run(result,feed_dict={a:[1.0,2.0],b:[3.0,4.0]})
    print(result)

Tensor("add:0", shape=(2,), dtype=float32)


placeholer机制

Tensorflow提供了placeholder机制，用于在会话运行时动态提供输入数据。编程时只需要将数据通过placeholder传入Tensorflow计算图即可。

之前我们将输入a和b定义为常量，这里将它们定义为一个placeholder，在运行会话时需要通过Session.run()函数的feed_dict参数来提供a和b的值。

feed_dict是一个字典(Dictionary),在字典中需要给出每个用到的palceholder的取值。如果参与运算的palceholder没有指定取值，那么程序就会报错

In [5]:
a = tf.placeholder(tf.float32,shape=(2),name='input')
b = tf.placeholder(tf.float32,shape=(2),name='input')
result = a + b

with tf.Session().as_default() as sess:
    sess.run(result,feed_dict={a:[1.0,2.0]})#这里没有提供b的值，所以报错。
    print(result)

InvalidArgumentError: You must feed a value for placeholder tensor 'input_3' with dtype float and shape [2]
	 [[node input_3 (defined at <ipython-input-5-44cd4502a678>:2) ]]
	 [[node add_1 (defined at <ipython-input-5-44cd4502a678>:3) ]]

Caused by op 'input_3', defined at:
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "E:\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tornado\platform\asyncio.py", line 149, in st

placehoder的目的是为了解决如何在有限的输入节点上实现高效地接受大量数据的问题。
在上面的样例程序中，如果将输入的b从长度为2的向量改成大小为n*2的矩阵，也就是可以得到n个向量相加的结果了，结果矩阵的每一行就代表了一个向量相加的结果。

In [8]:
a = tf.placeholder(tf.float32,shape=(2),name='input')
b = tf.placeholder(tf.float32,shape=(4,2),name='input')
result = a+b

with tf.Session() as sess:
    print(sess.run(result,feed_dict={a:[1.0,2.0],b:[[2.0,4.0],[5.0,6.0],[7.0,8.0],[9.0,10.0]]}))
    print(result)

[[ 3.  6.]
 [ 6.  8.]
 [ 8. 10.]
 [10. 12.]]
Tensor("add_3:0", shape=(4, 2), dtype=float32)


上述样例展示了placeholder的这种特性，我们不必定义4个b来用作网格的输入。在运行时，把4个样例[2.0,4.0],[5.0,6.0],[7.0,8.0],[9.0,10.0]组成一个4*2的矩阵传入palceholder,计算得到的结果为4*2的矩阵。